# Backtest

Backtest for simple rsi vs buy and hold strat

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the Excel file
file_path = '../../data/raw/Index Prices.xlsx'
xls = pd.ExcelFile(file_path)

# Load the data from each sheet
data_xbi = pd.read_excel(xls, sheet_name='XBI')
data_spx = pd.read_excel(xls, sheet_name='SPX')
data_nbi = pd.read_excel(xls, sheet_name='NBI')
data_m1wo0hc = pd.read_excel(xls, sheet_name='M1WO0HC')

In [3]:
def calculate_rsi(data, date_column, price_column, output_path, period=14, output_index=False):
    delta = data[price_column].diff(1)  # Calculate the difference between the current and previous close
    gain = delta.where(delta > 0, 0)  # Keep only gains
    loss = -delta.where(delta < 0, 0)  # Keep only losses (in positive values)
    
    avg_gain = gain.rolling(window=period, min_periods=period).mean()  # Rolling average gain
    avg_loss = loss.rolling(window=period, min_periods=period).mean()  # Rolling average loss
    
    rs = avg_gain / avg_loss  # Relative Strength (RS)
    rsi = 100 - (100 / (1 + rs))  # Relative Strength Index (RSI)
    
    # Create a DataFrame to store all the calculations, including Date
    rsi_data = pd.DataFrame({
        'Date': data[date_column],
        'Close': data[price_column],
        'Change': delta,
        'Gain': gain,
        'Loss': loss,
        'Avg Gain': avg_gain,
        'Avg Loss': avg_loss,
        'RS': rs,
        'RSI 14': rsi  
    })
    
    # Also output csv for further inspection
    rsi_data.to_csv(output_path, index=output_index)
    return rsi_data

# Calculate RSI and other metrics for each index
data_xbi_rsi = calculate_rsi(data_xbi, 'Unnamed: 0', 'XBI Equity', 'generated_data/XBI.csv')
data_spx_rsi = calculate_rsi(data_spx, 'Unnamed: 0', 'SPX Index', 'generated_data/SPX.csv')
data_nbi_rsi = calculate_rsi(data_nbi, 'Unnamed: 0', 'NBI Index', 'generated_data/NBI.csv')
data_m1wo0hc_rsi = calculate_rsi(data_m1wo0hc, 'Unnamed: 0', 'M1WO0HC Index', 'generated_data/M1WO0HC.csv')

In [4]:
def rsi_strategy(data, rsi_column='RSI 14', price_column='Close', date_column = 'Date', rsi_overbought=70, rsi_oversold=30):
    signals = []
    position = 0  # 0 means no position, 1 means holding a position
    for i in range(1, len(data)):
        if data[rsi_column].iloc[i] < rsi_oversold and position == 0:
            # Buy signal (Oversold condition)
            signals.append((data[date_column].iloc[i], 'Buy', data[price_column].iloc[i], data[rsi_column].iloc[i]))
            position = 1  # Enter position
        elif data[rsi_column].iloc[i] > rsi_overbought and position == 1:
            # Sell signal (Overbought condition)
            signals.append((data[date_column].iloc[i], 'Sell', data[price_column].iloc[i], data[rsi_column].iloc[i]))
            position = 0  # Exit position
    return signals

# Apply the RSI strategy to each index
signals_xbi= rsi_strategy(data_xbi_rsi)
signals_spx= rsi_strategy(data_spx_rsi)
signals_nbi= rsi_strategy(data_nbi_rsi)
signals_m1wo0hc = rsi_strategy(data_m1wo0hc_rsi)

In [5]:
def buy_and_hold_performance(data, price_column='Close'):
    buy_price = data[price_column].iloc[0]  # First price
    sell_price = data[price_column].iloc[-1]  # Last price
    total_return = sell_price - buy_price
    total_return_pct = (total_return) / buy_price * 100
    return total_return, total_return_pct

# Calculate buy-and-hold performance data for each index
buy_and_hold_profit_xbi, buy_and_hold_profit_pct_xbi = buy_and_hold_performance(data_xbi_rsi)
buy_and_hold_profit_spx, buy_and_hold_profit_pct_spx = buy_and_hold_performance(data_spx_rsi)
buy_and_hold_profit_nbi, buy_and_hold_profit_pct_nbi = buy_and_hold_performance(data_nbi_rsi)
buy_and_hold_profit_m1wo0hc, buy_and_hold_profit_pct_m1wo0hc = buy_and_hold_performance(data_m1wo0hc_rsi)

In [6]:
def performance_analysis(signals, output_path, index_name, buy_and_hold_profit, buy_and_hold_profit_pct):
    # Convert signals to a DataFrame
    signals_df = pd.DataFrame(signals, columns=['Date', 'Type', 'Price', 'RSI'])
    
    # Separate buy and sell signals
    buys = signals_df[signals_df['Type'] == 'Buy'].reset_index(drop=True)
    sells = signals_df[signals_df['Type'] == 'Sell'].reset_index(drop=True)
    
    # Ensure matching buy-sell pairs
    trades = pd.DataFrame({
        'Buy Date': buys['Date'],
        'Buy Price': buys['Price'],
        'Buy RSI': buys['RSI'],
        'Sell Date': sells['Date'],
        'Sell Price': sells['Price'],
        'Sell RSI': sells['RSI']
    })

    # Calculate trade return and true positive flags
    trades['Trade Return ($)'] = trades['Sell Price'] - trades['Buy Price']
    trades['Trade Return (%)'] = (trades['Trade Return ($)'] / trades['Buy Price']) * 100
    trades['True Positive'] = trades['Trade Return ($)'] > 0

    # Count true positives and false positives
    true_positives = trades['True Positive'].sum()
    false_positives = len(trades) - true_positives
    
    # Calculate profits and losses
    profits = trades.loc[trades['True Positive'], 'Trade Return ($)']
    losses = trades.loc[~trades['True Positive'], 'Trade Return ($)']
    
    total_profit = trades['Trade Return ($)'].sum()
    avg_profit = profits.mean() if not profits.empty else 0
    avg_loss = losses.mean() if not losses.empty else 0
    
    initial_capital = trades['Buy Price'].iloc[0] if not trades.empty else 0

    total_trades = true_positives + false_positives
    true_positive_ratio = true_positives / total_trades if total_trades > 0 else 0

    expected_return = (true_positive_ratio * avg_profit) + ((1 - true_positive_ratio) * avg_loss)
    
    # Convert dollar amounts to percentages
    total_profit_pct = (total_profit / initial_capital) * 100 if initial_capital else 0
    avg_profit_pct = (avg_profit / initial_capital) * 100 if initial_capital else 0
    avg_loss_pct = (avg_loss / initial_capital) * 100 if initial_capital else 0
    expected_return_pct = (expected_return / initial_capital) * 100 if initial_capital else 0

    # Save trades to CSV
    trades.to_csv(output_path, index=False)
    print("="*40)
    # Print the results directly within the function
    print(f"{index_name} Results")
    print("="*40)

    # Section for RSI Strategy
    print("RSI Strategy Performance:")
    print('-'*25)
    print(f"Total Profit: ${total_profit:.2f} ({total_profit_pct:.2f}%)")
    print(f"Average Profit (True Positive): ${avg_profit:.2f} ({avg_profit_pct:.2f}%)")
    print(f"Average Loss (False Positive): ${avg_loss:.2f} ({avg_loss_pct:.2f}%)")
    print(f"True Positive Ratio: {true_positive_ratio:.2f}")
    print(f"Expected Return: ${expected_return:.2f} ({expected_return_pct:.2f}%)\n")

    # Section for Buy-and-Hold Strategy
    print("Buy-and-Hold Strategy Performance")
    print('-'*25)

    print(f"Total Profit: ${buy_and_hold_profit:.2f} ({buy_and_hold_profit_pct:.2f}%)\n")
    # Uncomment the following line to return the computed values for further analysis. 
    # return (total_profit, total_profit_pct), (avg_profit, avg_profit_pct), (avg_loss, avg_loss_pct), initial_capital, (true_positives, false_positives, true_positive_ratio), (expected_return, expected_return_pct)

In [7]:
# Perform analysis and print results for XBI
performance_analysis(signals_xbi, 'trades/XBI_trades.csv', "XBI", buy_and_hold_profit_xbi, buy_and_hold_profit_pct_xbi)


XBI Results
RSI Strategy Performance:
-------------------------
Total Profit: $3.70 (5.37%)
Average Profit (True Positive): $6.75 (9.81%)
Average Loss (False Positive): $-12.19 (-17.72%)
True Positive Ratio: 0.65
Expected Return: $0.16 (0.23%)

Buy-and-Hold Strategy Performance
-------------------------
Total Profit: $49.67 (99.16%)



In [8]:
# Perform analysis and print results for SPX
performance_analysis(signals_spx, 'trades/SPX_trades.csv', "SPX", buy_and_hold_profit_spx, buy_and_hold_profit_pct_spx)

SPX Results
RSI Strategy Performance:
-------------------------
Total Profit: $2058.72 (109.64%)
Average Profit (True Positive): $135.41 (7.21%)
Average Loss (False Positive): $-230.05 (-12.25%)
True Positive Ratio: 0.85
Expected Return: $79.18 (4.22%)

Buy-and-Hold Strategy Performance
-------------------------
Total Profit: $3568.73 (179.60%)



In [9]:
# Perform analysis and print results for NBI
performance_analysis(signals_nbi, 'trades/NBI_trades.csv', "NBI", buy_and_hold_profit_nbi, buy_and_hold_profit_pct_nbi)

NBI Results
RSI Strategy Performance:
-------------------------
Total Profit: $1201.22 (44.95%)
Average Profit (True Positive): $182.46 (6.83%)
Average Loss (False Positive): $-214.76 (-8.04%)
True Positive Ratio: 0.67
Expected Return: $50.05 (1.87%)

Buy-and-Hold Strategy Performance
-------------------------
Total Profit: $2105.05 (77.68%)



In [10]:
# Perform analysis and print results for M1WO0HC
performance_analysis(signals_m1wo0hc, 'trades/M1WO0HC_trades.csv', "M1WO0HC", buy_and_hold_profit_m1wo0hc, buy_and_hold_profit_pct_m1wo0hc)

M1WO0HC Results
RSI Strategy Performance:
-------------------------
Total Profit: $187.83 (80.56%)
Average Profit (True Positive): $11.28 (4.84%)
Average Loss (False Positive): $-10.07 (-4.32%)
True Positive Ratio: 0.79
Expected Return: $6.71 (2.88%)

Buy-and-Hold Strategy Performance
-------------------------
Total Profit: $319.57 (133.04%)

